In [ ]:
# 1. Knowledge Cohort(knowledge extract):Knowledge Extraction
import os, sys, logging, autogen
from configs import GPT_MINI_CONFIG, MAX_ROUND 
try:
    from CeProAgents import KnowledgeGroup
except ImportError:
    sys.path.append(os.path.abspath("..")) 
    from CeProAgents import KnowledgeGroup

INPUT_DIR = "./CeProBench/knowledge/knowledge_raw" 
WORK_DIR = "./workdir/knowledge_extract"            
TARGET_ID = 1                                       
LLM_CONFIG = GPT_MINI_CONFIG                        

def get_file_path(base_dir, file_id):
    if not os.path.exists(base_dir): return None
    for f in os.listdir(base_dir):
        if f.startswith(f"{file_id}_"): return os.path.join(base_dir, f)
    for ext in ['.txt', '.md', '.json', '.pdf']:
        path = os.path.join(base_dir, f"{file_id}{ext}")
        if os.path.exists(path): return path
    return None

input_path = get_file_path(INPUT_DIR, TARGET_ID)

if input_path:
    file_name = os.path.basename(input_path)
    output_dir = os.path.abspath(os.path.join(WORK_DIR, "demo_run", os.path.splitext(file_name)[0]))
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"🚀 [Start] Ingesting: {file_name}")
    print(f"📂 [Target] Saving to: {output_dir}")

    manager = KnowledgeGroup(LLM_CONFIG, MAX_ROUND).get_manager()
    user = autogen.UserProxyAgent(name="User", human_input_mode="NEVER", code_execution_config=False, max_consecutive_auto_reply=1)

    task_msg = f"""Command: INGEST
        Source File: "{os.path.abspath(input_path)}"
        Target Directory: "{output_dir}"

        Instructions:
        1. Read the Source File.
        2. Build the knowledge base.
        3. Save index/data files to the Target Directory."""

    user.initiate_chat(manager, message=task_msg)
    print(f"✅ [Done] Task finished for ID {TARGET_ID}")
else:
    print(f"❌ [Error] File with ID {TARGET_ID} not found in {INPUT_DIR}")

In [ ]:
# 2. Knowledge Cohort(knowledge augment): Multi-Agent QA
import os, json, logging, autogen
from configs import GPT_CONFIG, MAX_ROUND
try:
    from CeProAgents import KnowledgeGroup
except ImportError:
    import sys
    sys.path.append(os.path.abspath(".."))
    from CeProAgents import KnowledgeGroup

WORK_DIR = "./workdir/knowledge_augment"
LLM_CONFIG = GPT_CONFIG  

DEMO_TASK = {
    "id": 1,
    "class": "General",
    "question": "What are the key safety considerations for high-pressure hydrogen storage?", 
    "answer": "N/A (Demo)" 
}

print(f"🧐 [Question ID {DEMO_TASK['id']}] Processing: {DEMO_TASK['question']}")

knowledge_group = KnowledgeGroup(LLM_CONFIG, MAX_ROUND)
manager = knowledge_group.get_manager()

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config=False,
    max_consecutive_auto_reply=1
)

task_payload = f"""Command: Query Task
Context:
- Query: "{DEMO_TASK['question']}"

Instructions:
Knowledge Augment(KG, RAG and Web) based on this query and then give the final answer."""

try:
    chat_result = user.initiate_chat(
        manager,
        message=task_payload,
        summary_method="reflection_with_llm", 
        summary_args={"summary_prompt": "Output the final report directly."}
    )
    final_answer = chat_result.summary
    print(f"\n✅ [Prediction]:\n{final_answer}\n")

    if not os.path.exists(WORK_DIR): os.makedirs(WORK_DIR)
    output_file = os.path.join(WORK_DIR, "demo_qa_result.jsonl")
    
    result_item = {**DEMO_TASK, "prediction": final_answer}
    
    with open(output_file, 'a', encoding='utf-8') as f:
        f.write(json.dumps(result_item, ensure_ascii=False) + "\n")s
    print(f"📂 Result saved to: {output_file}")

except Exception as e:
    print(f"❌ Error: {str(e)}")

In [ ]:
#3. Concept Cohort: P&ID Parsing
import os, sys, autogen
from configs import GPT_MINI_CONFIG, GEMINI_MINI_CONFIG 
try:
    from CeProAgents import ConceptGroup
    from CeProAgents.groups import clean_and_parse_json, save_json_file
except ImportError:
    sys.path.append(os.path.abspath(".."))
    from CeProAgents import ConceptGroup
    from CeProAgents.groups import clean_and_parse_json, save_json_file

INPUT_DIR = "./CeProBench/concept/PID_image"      
OUTPUT_DIR = "./results/concept/parse_demo"       
TARGET_ID = 1                                     
LLM_CONFIG = GPT_MINI_CONFIG                       

input_image_path = os.path.abspath(os.path.join(INPUT_DIR, f"{TARGET_ID}.png"))
output_json_path = os.path.join(OUTPUT_DIR, f"{TARGET_ID}_parse.json")

if os.path.exists(input_image_path):
    print(f"🖼️ [Processing Image]: {input_image_path}")
    
    concept_group = ConceptGroup(LLM_CONFIG, current_mode="parsing_only")
    manager = concept_group.get_manager()
    
    user = autogen.UserProxyAgent(
        name="User",
        human_input_mode="NEVER",
        code_execution_config=False,
        max_consecutive_auto_reply=1
    )

    task_payload = f"""
    **Task Type:** The Parsing Phase
    **Instruction:** Parse the P&ID image at the following path: "{input_image_path}"
    Call the parsing tool and return the JSON data.
    
    Example format (Reference only):
    ```json
    {{
      "equipments": [{{"identifier": "共沸剂冷凝器: E0301", "type" : "冷却/冷凝"}}], 
      "connections": [{{"source": "E0303", "target" : "E0401"}}]
    }}
    ```
    """

    try:
        chat_result = user.initiate_chat(
            manager,
            message=task_payload,
            summary_method="reflection_with_llm",
            summary_args={
                "summary_prompt": "Find the final JSON extracted from the image. Return ONLY the JSON code block."
            }
        )
        
        content = chat_result.summary
        if content:
            pid_json = clean_and_parse_json(content) 
            if pid_json:
                if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)
                save_json_file(pid_json, output_json_path)
                print(f"✅ [Success] Parsed data saved to: {output_json_path}")
                print(f"📊 [Preview] Equipments found: {len(pid_json.get('equipments', []))}")
            else:
                print("⚠️ [Warn] Failed to parse JSON content from response.")
        else:
            print("❌ [Error] No response content received.")

    except Exception as e:
        print(f"❌ [Exception]: {str(e)}")
else:
    print(f"❌ [Error] Image not found: {input_image_path}")

In [ ]:
#4. Concept Cohort: Topology Completion 
import os, sys, autogen
from configs import GPT_MINI_CONFIG

try:
    from CeProAgents import ConceptGroup
    from CeProAgents.groups import clean_and_parse_json, save_json_file
except ImportError:
    sys.path.append(os.path.abspath(".."))
    from CeProAgents import ConceptGroup
    from CeProAgents.groups import clean_and_parse_json, save_json_file

INPUT_DIR = "./CeProBench/concept/PID_complete"  
OUTPUT_DIR = "./results/concept/completion_demo" 
TARGET_ID = 1                                    
LLM_CONFIG = GPT_MINI_CONFIG                    

input_image_path = os.path.abspath(os.path.join(INPUT_DIR, f"{TARGET_ID}_mask_new.png"))
output_json_path = os.path.join(OUTPUT_DIR, f"{TARGET_ID}_completion.json")

if os.path.exists(input_image_path):
    print(f"🧩 [Processing Masked Image]: {input_image_path}")
    
    concept_group = ConceptGroup(LLM_CONFIG, current_mode="completion")
    manager = concept_group.get_manager()
    
    user = autogen.UserProxyAgent(
        name="User",
        human_input_mode="NEVER",
        code_execution_config=False,
        max_consecutive_auto_reply=1
    )
    task_payload = f"""
    **Task Type:** The Completion Phase
    
    **Instruction:** 
    The P&ID image at the following path contains a **masked area** (white box or occlusion): 
    "{input_image_path}"
    
    Your goal is to infer the missing process logic and reconstruct the P&ID structure.
    
    **Steps:**
    1. **Analyze Context:** Observe the pipelines and signals entering and exiting the masked area.
    2. **Infer Logic:** Based on chemical engineering principles, deduce what equipments or connections are missing.
    
    **Output Requirement:**
    - DO NOT replace the word 'mask' in the equipments list.
    - Provide a "completion" field with a Top 10 ranking of likely equipment types.
    
    Example Schema:
    ```json
    {{
        "equipments": [...],
        "connections": [...],
        "completion": [
            "Most Likely Type", "2nd Most Likely", ..., "10th Most Likely"
        ]
    }}
    ```
    """

    try:
        user.initiate_chat(manager, message=task_payload)
        
        real_content = None
        for msg in reversed(concept_group.groupchat.messages):
            content = msg.get("content")
            if msg.get("name") == "completer_expert" and "completion" in str(content):
                real_content = content
                break
        
        if real_content:
            pid_json = clean_and_parse_json(real_content)
            if pid_json:
                if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)
                save_json_file(pid_json, output_json_path)
                print(f"✅ [Success] Completion result saved to: {output_json_path}")
                print(f"🏆 [Top Prediction]: {pid_json.get('completion', [])[:3]} ...")
            else:
                print("⚠️ [Warn] JSON parsing failed.")
        else:
            print("❌ [Error] No valid completion output found in chat history.")

    except Exception as e:
        print(f"❌ [Exception]: {str(e)}")
else:
    print(f"❌ [Error] Masked image not found: {input_image_path}")

In [ ]:
# 5. Concept Cohort: Generative Design
import os, glob, json, autogen
from configs import GPT_MINI_CONFIG

try:
    from CeProAgents import ConceptGroup
    from CeProAgents.groups import clean_and_parse_json, save_json_file
except ImportError:
    import sys
    sys.path.append(os.path.abspath(".."))
    from CeProAgents import ConceptGroup
    from CeProAgents.groups import clean_and_parse_json, save_json_file

INPUT_DIR = "./CeProBench/concept/PID_generate"    
OUTPUT_DIR = "./results/concept/generation_demo"  
LLM_CONFIG = GPT_MINI_CONFIG                      

if not os.path.exists(INPUT_DIR): os.makedirs(INPUT_DIR)
demo_prompt_path = os.path.join(INPUT_DIR, "demo_case.txt")
if not os.path.exists(demo_prompt_path):
    with open(demo_prompt_path, "w", encoding="utf-8") as f:
        f.write("Design a simple P&ID for a water tank level control system. It should include a water tank, a feed pump, a level sensor, and a control valve.")

def process_single_prompt(concept_manager, user_proxy, file_path, output_dir):
    file_basename = os.path.splitext(os.path.basename(file_path))[0]
    print(f"📄 [Processing Prompt]: {file_basename}")

    with open(file_path, 'r', encoding='utf-8') as f:
        prompt_content = f.read().strip()

    task_payload = f"""
    **Task Type:** P&ID Generation
    **Input Description:** 
    "{prompt_content}"

    **Instruction:** 
    Based on the description above, design the P&ID and output the result in structured JSON format.
    The JSON must adhere to the schema: {{ "equipments": [...], "connections": [...] }}
    """

    try:
        user_proxy.initiate_chat(concept_manager, message=task_payload, summary_method=None)

        history = concept_manager.groupchat.messages
        first_draft = None
        final_draft = None

        for msg in history:
            if msg.get("name") == "generator_expert":
                content = msg.get("content", "")
                parsed = clean_and_parse_json(content)
                if parsed:
                    if first_draft is None: first_draft = parsed
                    final_draft = parsed 
        if not os.path.exists(output_dir): os.makedirs(output_dir)
        
        if first_draft:
            save_json_file(first_draft, os.path.join(output_dir, f"{file_basename}_first.json"))
            print(f"  📝 [First Draft] Saved.")
            
        if final_draft:
            save_json_file(final_draft, os.path.join(output_dir, f"{file_basename}_final.json"))
            print(f"  ✅ [Final Result] Saved.")
            print(f"  📊 [Stats] Equipments: {len(final_draft.get('equipments', []))}, Connections: {len(final_draft.get('connections', []))}")
        else:
            print(f"  ❌ [Error] No valid JSON generated.")

    except Exception as e:
        print(f"  ❌ [Exception]: {str(e)}")

concept_group = ConceptGroup(LLM_CONFIG, current_mode="generation")
concept_manager = concept_group.get_manager()
user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    max_consecutive_auto_reply=1,
)

txt_files = glob.glob(os.path.join(INPUT_DIR, "*.txt"))
print(f"📂 Found {len(txt_files)} prompt files.")

for file_path in txt_files:
    concept_manager.reset()
    user_proxy.reset()
    process_single_prompt(concept_manager, user_proxy, file_path, OUTPUT_DIR)

In [ ]:
# 6. Parameter Cohort: Simulation Optimization 
import os, json, logging, autogen
from configs import CLAUDE_CONFIG, GPT_CONFIG
try:
    from CeProAgents import SimulationGroup
    from CeProAgents.groups.parameter_group.aspen_utils import extract_aspen_block_set_parameters
    from CeProAgents.groups.parameter_group.simulation_utils import extract_aspen_flowsheet_connections
except ImportError:
    import sys
    sys.path.append(os.path.abspath(".."))
    from CeProAgents import SimulationGroup
    from CeProAgents.groups.parameter_group.aspen_utils import extract_aspen_block_set_parameters
    from CeProAgents.groups.parameter_group.simulation_utils import extract_aspen_flowsheet_connections

INPUT_DIR = "./CeProBench/parameter/cases"        
OUTPUT_DIR = "./results/parameter/optimization"    
CASE_NAME = "distillation_demo"                   
LLM_CONFIG = CLAUDE_CONFIG                         

bkp_path = os.path.abspath(os.path.join(INPUT_DIR, f"{CASE_NAME}.bkp"))
goal_path = os.path.abspath(os.path.join(INPUT_DIR, f"{CASE_NAME}.txt"))
if os.path.exists(bkp_path) and os.path.exists(goal_path):
    print(f"🏭 [Starting Optimization]: {CASE_NAME}")
    print(f"📄 BKP File: {bkp_path}")
    
    try:
        with open(goal_path, 'r', encoding='utf-8') as f:
            goal_content = f.read().strip()
        print("⏳ Extracting Aspen Plus data (Connect via COM)...")
        pid_data = extract_aspen_flowsheet_connections(bkp_path)
        param_data = extract_aspen_block_set_parameters(bkp_path)
        print("✅ Extraction complete.")

        global_context = {
            "bkp_file_path": bkp_path,
            "pid_json": pid_data,
            "optimization_goal": goal_content
        }
        sim_group = SimulationGroup(llm_config=LLM_CONFIG, global_context=global_context)
        manager = sim_group.get_manager()

        user = autogen.UserProxyAgent(
            name="User_Proxy",
            human_input_mode="NEVER",
            code_execution_config=False,
            max_consecutive_auto_reply=1
        )

        task_payload = f"""
        **Mission**: Optimize simulation parameters for project: {CASE_NAME}
        **Context**:
        - bkp_file_path: "{bkp_path}"
        - optimization_goal: "{goal_content}"
        - pid_json: {json.dumps(pid_data, ensure_ascii=False)}
        - param_json: {json.dumps(param_data, ensure_ascii=False)}
        """

        chat_result = user.initiate_chat(
            manager,
            message=task_payload,
            summary_method="reflection_with_llm",
            summary_args={"summary_prompt": "Extract the FINAL optimized parameters and metrics as a clean JSON object."}
        )
        if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)
        
        summary_file = os.path.join(OUTPUT_DIR, f"{CASE_NAME}_summary.json")
        with open(summary_file, 'w', encoding='utf-8') as f:
            f.write(chat_result.summary)
            
        history_file = os.path.join(OUTPUT_DIR, f"{CASE_NAME}_history.json")
        with open(history_file, 'w', encoding='utf-8') as f:
            json.dump(chat_result.chat_history, f, indent=2, ensure_ascii=False)
            
        print(f"🎉 Optimization Finished! Results saved to: {OUTPUT_DIR}")

    except Exception as e:
        print(f"❌ [Error]: {str(e)}")
        print("💡 Hint: Ensure Aspen Plus V11+ is installed and the license is active.")
else:
    print(f"⚠️ [Skip] Files not found. Please ensure {CASE_NAME}.bkp and .txt exist in {INPUT_DIR}")